# Part 1

In [2]:
with open('example.txt', 'r') as f:
    lines = [entry.strip() for entry in f.readlines()]

In [3]:
calling_numbers = [int(entry) for entry in lines[0].strip().split(',')]

In [1]:
import numpy as np

In [16]:
def read_board(lines):
    board = np.zeros((5,5))
    for i in range(5):
        line_entries = [int(entry) for entry in lines[i].split(' ') if entry != '']
        board[i] = line_entries
    return board

In [5]:
lines

['7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1',
 '',
 '22 13 17 11  0',
 '8  2 23  4 24',
 '21  9 14 16  7',
 '6 10  3 18  5',
 '1 12 20 15 19',
 '',
 '3 15  0  2 22',
 '9 18 13 17  5',
 '19  8  7 25 23',
 '20 11 10 24  4',
 '14 21 16 12  6',
 '',
 '14 21 17 24  4',
 '10 16 15  9 19',
 '18  8 23 26 20',
 '22 11 13  6  5',
 '2  0 12  3  7']

In [15]:
test_a = '8  2 23  4 24'
test_a.split(' ')

['8', '', '2', '23', '', '4', '24']

In [17]:
number_of_boards = (len(lines)-1)//6
print(number_of_boards)
boards = dict()
# read boards
for j in range(number_of_boards):
    boards[j] = read_board(lines[(2 + j*6):(2+5+(j+1)*6)])

3


In [22]:
boards[0]

array([[22., 13., 17., 11.,  0.],
       [ 8.,  2., 23.,  4., 24.],
       [21.,  9., 14., 16.,  7.],
       [ 6., 10.,  3., 18.,  5.],
       [ 1., 12., 20., 15., 19.]])

In [25]:
25 in boards[0]

False

In [31]:
indices = np.where(boards[0] == 24)
indices

(array([1]), array([4]))

In [33]:
boards[0][indices[0], indices[1]]

array([24.])

In [46]:
check_array = (boards[0] > 3) * 1.0
check_array

array([[1., 1., 1., 1., 0.],
       [1., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [0., 1., 1., 1., 1.]])

In [47]:
check_array.all(axis=0)

array([False, False, False,  True, False])

Refactoring and adding additional methods to the Board class, using the above logic to read in the board numbers.

In [69]:
class Board:
    def __init__(self):
        self.board = np.zeros((5,5))
        self.marked = np.zeros((5,5))

    def read_from_lines(self, lines):
        for i in range(5):
            line_entries = [int(entry) for entry in lines[i].split(' ') if entry != '']
            self.board[i] = line_entries
    
    def check_called_number(self, called_number):
        if called_number in self.board:
            indices = np.where(self.board == called_number)
            self.marked[indices[0], indices[1]] = 1
    
    def check_win(self):
        # check rows, check columns
        return self.marked.all(axis=1).any() or self.marked.all(axis=0).any()

    def calculate_score(self, called_number):
        # sum of all unmarked numbers on that board; 
        # multiply that sum by the number that was just called
        return (self.board * (self.marked == 0)).sum() * called_number

In [77]:
def find_first_winner():
    for called_number in calling_numbers:
        # print('called number', called_number)
        for j in range(number_of_boards):
            boards[j].check_called_number(called_number)
            if boards[j].check_win():
                print(f"Board {j} won!")
                print(boards[j].marked)
                return j, called_number

In [75]:
number_of_boards = (len(lines)-1)//6
print(number_of_boards)
boards = dict()
# read boards
for j in range(number_of_boards):
    boards[j] = Board()
    boards[j].read_from_lines(lines[2 + j*6 : 2+5+ (j+1)*6 ])
winner_index, called_number = find_first_winner()
final_score = boards[winner_index].calculate_score(called_number)
print('final score', final_score)

3
Board 2 won!
[[1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1.]
 [1. 1. 0. 0. 1.]]
final score 4512.0


Code works for example input, now let's try on the real input:

In [76]:
with open('input.txt', 'r') as f:
    lines = [entry.strip() for entry in f.readlines()]

calling_numbers = [int(entry) for entry in lines[0].strip().split(',')]

number_of_boards = (len(lines)-1)//6
print(number_of_boards)
boards = dict()
# read boards
for j in range(number_of_boards):
    boards[j] = Board()
    boards[j].read_from_lines(lines[2 + j*6 : 2+5+ (j+1)*6 ])
winner_index, called_number = find_first_winner()
final_score = boards[winner_index].calculate_score(called_number)
print('final score', final_score)

100
Board 60 won!
[[0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1.]
 [0. 1. 0. 0. 0.]]
final score 25023.0


Yay, correct!! :)

# Part 2

figure out which board will win last - first, again, with the example input, where we are told the solution is the second board (index 1)

We need to adjust the ''find_first_winner'' function, so it now will show us the last winner. This means, we need to keep track of the winners.

We also can't break anymore after finding the winner, because we need to look for further winners. That means we don't know when to break and the strategy of calculating the score after returning from the function is wrong, as at that point more points on the board are marked than at winning time.

So I calculate the last winners score in the for loop.

In [81]:
def find_last_winner():
    winners = []
    score = 0
    for called_number in calling_numbers:
        # print('called number', called_number)
        for j in range(number_of_boards):
            boards[j].check_called_number(called_number)
            if boards[j].check_win() and j not in winners:
                winners.append(j)
                score = boards[j].calculate_score(called_number)
    return winners[-1], score

In [83]:
with open('example.txt', 'r') as f:
    lines = [entry.strip() for entry in f.readlines()]

calling_numbers = [int(entry) for entry in lines[0].strip().split(',')]

number_of_boards = (len(lines)-1)//6
print(number_of_boards)
boards = dict()
# read boards
for j in range(number_of_boards):
    boards[j] = Board()
    boards[j].read_from_lines(lines[2 + j*6 : 2+5+ (j+1)*6 ])
winner_index, final_score = find_last_winner()
print('winner index', winner_index)
print('final score', final_score)

3
winner index 1
final score 1924.0


Correct answer for the example case, so now let's run it with the real input:

In [84]:
with open('input.txt', 'r') as f:
    lines = [entry.strip() for entry in f.readlines()]

calling_numbers = [int(entry) for entry in lines[0].strip().split(',')]

number_of_boards = (len(lines)-1)//6
print(number_of_boards)
boards = dict()
# read boards
for j in range(number_of_boards):
    boards[j] = Board()
    boards[j].read_from_lines(lines[2 + j*6 : 2+5+ (j+1)*6 ])
winner_index, final_score = find_last_winner()
print('winner index', winner_index)
print('final score', final_score)

100
winner index 21
final score 2634.0


Also correct! Yay :)